In [2]:
import sys
sys.path.append("..")
import os
os.environ['USE_PYGEOS'] = '0'
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols 
from rasterstats import zonal_stats
import rasterio
import contextily as cx
from rasterio.plot import show
#import osmnx as ox
from affine import Affine
import rioxarray as rx
from multiprocessing import Pool, cpu_count
from shapely.ops import unary_union
import numpy as np
from osgeo import gdal
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from collections import defaultdict
from pathlib import Path
import fiona



#see all columns in tables
pd.set_option('display.max_columns', None)


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [110]:
from src.features.parcel_prep import parcel_ms4_model

In [111]:
#list MAPC towns to include 
town_names = [
            'Reading', 'Winchester', 'Franklin', 'Natick', 'Norwood'
            ]

processed_path = "K:\\DataServices\\Projects\\Current_Projects\\Environment\\MS4\\Data\\Spatial\\Output"
model_path = "K:\\DataServices\\Projects\\Current_Projects\\Environment\\MS4\\Data\\Spatial\\Model"

In [ ]:
from src.features.build_features import calculate_suitability_criteria

def calculate_imperviousness(parcel_data, id_field, imprv_cover_layer, imprv_structure_layer):
    #calculate area of impervious cover
    imperv_surfaces = calculate_suitability_criteria(how='overlap_area', 
                                                id_field=id_field,
                                                parcel_data=parcel_data, 
                                                join_data=imprv_cover_layer, 
                                                field='type',
                                                layer_name='imp_cvr')
    #calculate area of rooftops
    imperv_rooftops = calculate_suitability_criteria(how='overlap_area', 
                                                    id_field=id_field,
                                                    parcel_data=parcel_data, 
                                                    join_data=imprv_structure_layer, 
                                                    field='type',
                                                    layer_name='imp_rf')
    #join together based on parloc id
    imperv = imperv_surfaces.merge(imperv_rooftops[[id_field, 'sqm_imp_rf', 'pct_imp_rf']], 
                                on=id_field, 
                                how='inner')
    
    imperv['sqm_imprv'] = imperv['sqm_imp_cvr'] + imperv['sqm_imp_rf']
    imperv['pct_imprv'] = imperv['sqm_imprv'] / imperv['geometry'].area
    imperv['sqm_prv'] = imperv['geometry'].area - imperv['sqm_imprv']
    imperv['pct_prv'] = 1 - imperv['pct_imprv']
    imperv.insert((len(imperv.columns) - 1), 'geometry', imperv.pop('geometry'))
    return(imperv)

In [113]:
from src.features.parcel_prep import parcel_ms4_model

#run suitability model for each town in list
for town_name in town_names:
    parcel_ms4_model(town_name)


# MERGE SHAPEFILES 

filelist = []


# add all reclassified rasters
for root, folder, files in os.walk(processed_path):
    for file in files:
        for muni in town_names:
            if muni in file:
                if file.endswith('.shp'):
                    fullname = os.path.join(root, file)
                    filelist.append(fullname)


merged_gdf = gpd.GeoDataFrame(pd.concat([gpd.read_file(i) for i in filelist], 
                        ignore_index=True), crs=gpd.read_file(filelist[0]).crs)


#export
merged_gdf.to_file(model_path + '\\Merged_MS4_Parcels.shp')

Validating land use geometry...
Validating imperviousness geometry...
Validating tree canopy geometry...
public land...
imperviousness...
imperviousness pler...
park serve need...
wetlands...
wpa...
aul...
watershed...
tree need...
Calculating heat vulnerability overlap...
Calculating ej block group overlap...
Validating land use geometry...
Validating imperviousness geometry...
Validating tree canopy geometry...
public land...
imperviousness...
imperviousness pler...
park serve need...
wetlands...
wpa...
aul...
watershed...
tree need...
Calculating heat vulnerability overlap...
Calculating ej block group overlap...
Validating land use geometry...
Validating imperviousness geometry...
Validating tree canopy geometry...
public land...
imperviousness...
imperviousness pler...
park serve need...
wetlands...
wpa...
aul...
watershed...
tree need...
Calculating heat vulnerability overlap...
Calculating ej block group overlap...
Validating land use geometry...
Validating imperviousness geomet

In [120]:
merged_gdf.loc[(merged_gdf['type'] == 'ROW segment') & (merged_gdf['pblc'] == 1)].head()

,parloc_id,Address,Owner,UseDesc,muni,type,acreage,USE_DESC,pblc,sqm_imp_cv,pct_imp_cv,sqm_imp_rf,pct_imp_rf,sqm_imprv,pct_imprv,sqm_prv,pct_prv,K_sum,K_per_acre,ParkRank,parks,IT_VALDESC,wtlnds,SUPPLIER_x,z1_wpa,SUPPLIER_y,z2_wpa,SUPPLIER,int_wpa,aul,NAME_x,NAME_y,wtshd,rnk_tree,tree,rnk_heat,heat,EJ_CRIT_DE,ej,hsg_scr_me,geometry
9984,ROW_10082,NaN,NaN,NaN,Franklin,ROW segment,NaN,"Vacant, Conservation (Municipal or County)",1,1.364098,0.000406,NaN,NaN,NaN,NaN,NaN,NaN,11.70000,14.076988,2.0,1,0,0,0,0,0,0,0,0,0,NaN,CHARLES,1,0.000000,0,0.000000,0,0,0,1.5,"POLYGON ((209143.679 873153.247, 209084.509 87..."
9987,ROW_10085,NaN,NaN,NaN,Franklin,ROW segment,NaN,"Vacant, Selectmen or City Council (Municipal)",1,3.161822,0.002885,NaN,NaN,NaN,NaN,NaN,NaN,3.90000,14.403087,3.0,1,WOODED SWAMP DECIDUOUS,1,0,0,0,0,0,0,0,NaN,CHARLES,1,0.000000,0,0.000000,0,0,0,1.0,"POLYGON ((209532.170 873012.615, 209531.270 87..."
9996,ROW_10094,NaN,NaN,NaN,Franklin,ROW segment,NaN,"Vacant, Selectmen or City Council (Municipal)",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1,0,0,0,0,0,0,0,0,0,NaN,CHARLES,1,0.210526,1,0.475248,1,0,0,1.0,"POLYGON ((206162.785 872493.051, 206163.156 87..."
10019,ROW_10117,NaN,NaN,NaN,Franklin,ROW segment,NaN,"Vacant, Selectmen or City Council (Municipal)",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,0,0,0,0,0,0,0,0,0,NaN,CHARLES,1,0.052632,1,0.680693,1,0,0,3.0,"POLYGON ((208419.896 871257.266, 208419.164 87..."
11433,ROW_11531,NaN,NaN,NaN,Franklin,ROW segment,NaN,"Vacant, Conservation (Municipal or County)",1,774.509632,0.587381,NaN,NaN,NaN,NaN,NaN,NaN,819.06002,2513.866288,2.0,1,0,0,0,0,0,0,0,0,0,NaN,CHARLES,1,0.000000,0,0.000000,0,0,0,1.5,"POLYGON ((209160.583 873155.760, 209143.679 87..."


In [3]:
from src.data.make_dataset import *
from src.features.build_features import *
from src.features.ms4_funcs import *

Reading in municipal and regional boundaries...
Reading in land parcel database data...
Reading in public excluded land data...
Reading in additional data layers...
Wetlands...
Watersheds...
Wellhead protection areas...
Activity use limitation areas...
ParkServe data...


### Identify muni/munis of interest, create a shapefile for the muni

Then create land use and land cover layers of interest based on the municipal boundary mask (speeds up processes)

### Pull base geographies for analysis
- Get municipal ROW and fishnet (tesselation) layers from geodatabases 
- Clean up those gdfs to include only the relevant info

`town_parcels_row` and `town_tess` are the two base geographies for this analysis.

## Inidicator Layer Prep 
To improve run time, for some indicators it is better to read in here with a municipal mask 

In [86]:
#select a town name, create a gdf of that town boundary
town_name = 'Franklin'

muni_shp = munis.loc[munis['municipal'] == town_name]
#read in relevant data layers
from shapely.validation import make_valid

row_gdb = 'K:\DataServices\Projects\Current_Projects\Environment\MS4\Project\ROW_model_output.gdb'

for layer_name in fiona.listlayers(row_gdb):
    if town_name in layer_name:
        town_row_layer = layer_name

town_row = gpd.read_file(row_gdb, layer=town_row_layer)
town_row = town_row.explode().reset_index(drop=True)

##FISHNET##

fishnets_gdb = 'K:\\\DataServices\\Projects\\Current_Projects\\Environment\\MS4\\Project\\MS4_Fishnets.gdb'

for layer_name in fiona.listlayers(fishnets_gdb):
    if town_name in layer_name:
        town_tess = layer_name

muni_tess = gpd.read_file(fishnets_gdb, layer=town_tess)
#muni_tess = gpd.read_file(muni_tess_gdb, layer='muni_tessellation')
#clip tesselation to the franklin boundary
muni_tess = muni_tess.clip(muni_shp)
muni_tess['sqm'] = muni_tess['geometry'].area

#create an imperviousness layer
#first read in lclu with a muni shapefile mask

ms4_model_gdb = 'K:\\DataServices\\Projects\\Current_Projects\\Environment\\MS4\\Project\\MS4_Model.gdb'
lclu = gpd.read_file(ms4_model_gdb, layer="lclu_simplify_all_mapc", mask=muni_shp)

#fix any geometry issues
print('Validating land use geometry...')
lclu.geometry = lclu.apply(lambda row: make_valid(row.geometry) if not row.geometry.is_valid else row.geometry, axis=1)
lclu = lclu.clip(muni_shp)

#get phosphorous load per land cover geometry area
lclu['acres'] = lclu['geometry'].area / 4047
lclu['K_load'] = lclu['pler'] * lclu['acres']

#select only for imperviousness land cover codes
lclu_impervious = lclu.loc[lclu['covercode'] == 2]

#fix any geometry issues
print('Validating imperviousness geometry...')
lclu_impervious.geometry = lclu_impervious.apply(lambda row: make_valid(row.geometry) if not row.geometry.is_valid else row.geometry, axis=1)

#tree canopy using land use
tree_canopy_covernames = ['Deciduous Forest', 'Evergreen Forest', 'Palustrine Forested Wetland']
lclu_treecanopy = lclu.loc[lclu['covername'].isin(tree_canopy_covernames)]

print('Validating tree canopy geometry...')
lclu_treecanopy.geometry = lclu_treecanopy.apply(lambda row: make_valid(row.geometry) if not row.geometry.is_valid else row.geometry, axis=1)

#fix geom inconsistnecies
lclu_treecanopy = lclu_treecanopy.explode()
lclu_treecanopy = lclu_treecanopy.loc[lclu_treecanopy['geometry'].geom_type == 'Polygon']

#make a layer that gives us impervious surfaces but distinguishes between impervious land cover vs rooftops

#read in structures for the muni
building_structures_gdb = 'K:\\DataServices\\Datasets\\MassGIS\\Facilities_Structures\\Building_Structures\\Output\\structures.gdb'
building_structures = gpd.read_file(building_structures_gdb, layer='STRUCTURES_POLY', mask=muni_shp)

#add a type field
building_structures['type'] = 'rooftop'

#erase structures from the imperviousness lclu layer
imperv_cover = lclu_impervious.overlay(building_structures, how='difference')

#now we just have land cover (not rooftops) that are imperviousness
#add a type field
imperv_cover['type'] = 'land cover'

#join back together with the 'type' field being the distinguisher btwn land cover and rooftops
imperviousness_by_type = pd.concat([building_structures[['type', 'geometry']],
                                    imperv_cover[['type', 'geometry']]
                                    ])

#dissolve by type
imperviousness_by_type = imperviousness_by_type.dissolve(by='type')

#add a "layer" field for the model
imperviousness_by_type['layer'] = 'impervious'

#get tree "need" layer (block group)
muni_tree_need = tree_score(muni_shp, lclu_treecanopy)
muni_tree_need.head()

#get heat "vulnerability" layer (block)
muni_heat_vuln = heat_score(muni_shp, heat_fp)

#get soil hydrology score
muni_soils_hsg = soil_hsg_score(muni_shp)


Validating land use geometry...
Validating imperviousness geometry...
Validating tree canopy geometry...


## Tesselation Analysis

In [114]:
#does it overlap with public land
print("Calculating public land overlap...")
public_tess = calculate_suitability_criteria(how='if_overlap', 
                                            id_field='GRID_ID',
                                            parcel_data=muni_tess, 
                                            join_data=public_land, 
                                            field='GROUP_',
                                            layer_name='pblc',
                                            overlap=0.05)

print("Calculating right of way overlap...")
town_row['ROW_ID'] = town_row['parloc_id']
row_tess = calculate_suitability_criteria(how='if_overlap', 
                                            id_field='GRID_ID',
                                            parcel_data=muni_tess, 
                                            join_data=town_row, 
                                            field='ROW_ID',
                                            layer_name='row',
                                            overlap=0.05)

print("Calculating imperviousness overlap...")
imperv_tess = calculate_imperviousness(muni_tess, 
                                  'GRID_ID', 
                                  imperv_cover, 
                                  building_structures)

print('Calculating pler...')
#what is the imperviousness pler on site?
pler_tess = get_K_load(parcel_data=muni_tess,
                         id_field = 'GRID_ID',
                         muni_name = town_name,
                         muni_shpf = muni_shp,
                         lclu_layer=lclu_impervious,
                         pler_field='pler')


print("Calculating park serve overlap...")
#does it overlap with Trust for Public Land's "Park Serve" priority areas from 2022
parkserve_tess = calculate_suitability_criteria(how='if_overlap', 
                                        id_field='GRID_ID',
                                        parcel_data=muni_tess, 
                                        join_data=parkserve_data, 
                                        field='ParkRank',
                                        layer_name='parks',
                                        overlap=0.05)

print("Calculating wetland overlap...")
#does it overlap with wetlands?
wetlands_ovlp_tess = calculate_suitability_criteria(how='if_overlap', 
                                            id_field='GRID_ID',
                                            parcel_data=muni_tess, 
                                            join_data=wetlands, 
                                            field='IT_VALDESC',
                                            layer_name='wtlnds',
                                            overlap=0.05)
print("Calculating wpa overlap...")
#does it overlap with wellhead protection areas?
wpa_z1_ovlp_tess = calculate_suitability_criteria(how='if_overlap', 
                                            id_field='GRID_ID',
                                            parcel_data=muni_tess, 
                                            join_data=zone1_wpa, 
                                            field='SUPPLIER',
                                            layer_name='z1_wpa',
                                            overlap=0.05)

wpa_z2_ovlp_tess = calculate_suitability_criteria(how='if_overlap', 
                                            id_field='GRID_ID',
                                            parcel_data=muni_tess, 
                                            join_data=zone2_wpa, 
                                            field='SUPPLIER',
                                            layer_name='z2_wpa',
                                            overlap=0.05)


wpa_interim_ovlp_tess = calculate_suitability_criteria(how='if_overlap', 
                                            id_field='GRID_ID',
                                            parcel_data=muni_tess, 
                                            join_data=interim_wpa, 
                                            field='SUPPLIER',
                                            layer_name='int_wpa',
                                            overlap=0.05)

print("Calculating aul overlap...")
#does it have an activity use limitation area within it?
aul_ovlp_tess = calculate_suitability_criteria(how='if_overlap', 
                                            id_field='GRID_ID',
                                            parcel_data=muni_tess, 
                                            points=True,
                                            join_data=aul, 
                                            field='NAME',
                                            layer_name='aul',
                                            overlap=0.05)

print("Calculating watershed overlap...")
#what watershed does it overlap with?
watershed_tess = calculate_suitability_criteria(how='overlap_sjoin', 
                                            id_field='GRID_ID',
                                            parcel_data=muni_tess, 
                                            join_data=major_basins, 
                                            stats='first',
                                            field='NAME',
                                            layer_name='wtshd')

print("Calculating tree need overlap...")
#is it in a block group with tree need?
tree_need_tess = calculate_suitability_criteria(how='if_overlap', 
                                        id_field='GRID_ID',
                                        parcel_data=muni_tess, 
                                        join_data=muni_tree_need, 
                                        field='rnk_tree',
                                        layer_name='tree',
                                        overlap=0.05)

print("Calculating heat vulnerability overlap...")
#is it in a census block with high heat vulnerability?
heat_vln_tess = calculate_suitability_criteria(how='if_overlap', 
                                        id_field='GRID_ID',
                                        parcel_data=muni_tess, 
                                        join_data=muni_heat_vuln, 
                                        field='rnk_heat',
                                        layer_name='heat',
                                        overlap=0.05)

print("Calculating ej block group overlap...")
#is it in an ej community?
ej_tess = calculate_suitability_criteria(how='if_overlap', 
                                        id_field='GRID_ID',
                                        parcel_data=muni_tess, 
                                        join_data=ej_2020, 
                                        field='EJ_CRIT_DESC',
                                        layer_name='ej',
                                        overlap=0.05)


Calculating public land overlap...
Calculating right of way overlap...
Calculating imperviousness overlap...
Calculating pler...
Calculating park serve overlap...
Calculating wetland overlap...
Calculating wpa overlap...
Calculating aul overlap...
Calculating watershed overlap...
Calculating tree need overlap...
Calculating heat vulnerability overlap...
Calculating ej block group overlap...


In [115]:

#then join parcel data to the tesselations. Each tessel may have more than one associated 
#this might be more relevant later
'''muni_tess_parcels = muni_tess.overlay(town_parcels_row[['parloc_id', 
                                                        'Address', 
                                                        'Owner', 
                                                        'UseDesc', 
                                                        'muni', 
                                                        'type', 
                                                        'acreage', 
                                                        'geometry']], 
                                                        how='intersection')'''
'''
muni_tess_parcels_agg = muni_tess_parcels[['GRID_ID', 'sqm', 'parloc_id', 'type']].groupby(
    ['GRID_ID'], as_index=False).agg(
        {'parloc_id':list, 'type':list})


muni_tess_parcels_agg = muni_tess.merge(muni_tess_parcels_agg, on='GRID_ID')'''

dfs = [ public_tess,
       row_tess,
       imperv_tess,
       pler_tess,
       parkserve_tess,
       wetlands_ovlp_tess,
       wpa_z1_ovlp_tess,
       wpa_z2_ovlp_tess,
       wpa_interim_ovlp_tess,
       aul_ovlp_tess,
       watershed_tess,
       tree_need_tess,
       heat_vln_tess,
       ej_tess
    ]

 #merge all of the listed dataframes together based on their parcel id and geometry
df_merged = reduce(lambda  left,right: pd.merge(left,
                                                right,
                                                on=['GRID_ID', 'geometry'], 
                                                how='outer'), 
                                                dfs)

df_merged.insert((len(df_merged.columns) - 1), 'geometry', df_merged.pop('geometry'))

#run suitability model for each town in list
processed_path = "K:\\DataServices\\Projects\\Current_Projects\\Environment\\MS4\\Data\\Spatial\\Output"

#add back merged shapefile (add this back later)

filelist = []

#export
df_merged.to_file(processed_path + '\\Test_Combined_Shapefile_MS4_gi_equity_tess.shp')
